In [26]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [28]:
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,6.686620,True,18,1,43.0
1,9.360828,True,32,1,41.0
2,8.437091,False,30,1,39.0
3,6.124442,False,24,1,40.0
4,7.125340,False,26,1,41.0


In [29]:
df = df.dropna()
df = shuffle(df, random_state=2)
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [30]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Train on 6670 samples, validate on 742 samples
Epoch 1/10
6670/6670 [==============================] - 2s 304us/sample - loss: 1.7531 - mae: 1.0002 - mse: 1.7531 - val_loss: 1.2598 - val_mae: 0.8782 - val_mse: 1.2598
Epoch 2/10
6670/6670 [==============================] - 1s 155us/sample - loss: 1.3741 - mae: 0.9203 - mse: 1.3741 - val_loss: 1.2605 - val_mae: 0.8915 - val_mse: 1.2605
Epoch 3/10
6670/6670 [==============================] - 1s 158us/sample - loss: 1.3428 - mae: 0.9104 - mse: 1.3428 - val_loss: 1.2146 - val_mae: 0.8730 - val_mse: 1.2146
Epoch 4/10
6670/6670 [==============================] - 1s 148us/sample - loss: 1.2847 - mae: 0.8858 - mse: 1.2847 - val_loss: 1.1022 - val_mae: 0.8220 - val_mse: 1.1022
Epoch 5/10
6670/6670 [==============================] - 1s 151us/sample - loss: 1.2857 - mae: 0.8900 - mse: 1.2857 - val_loss: 1.0852 - val_mae: 0.8227 - val_mse: 1.0852
Epoch 6/10
6670/6670 [==============================] - 1s 168us/sample - loss: 1.2640 - mae: 0.8867 - 

In [31]:
'''
num_examples = 10
predictions = model.predict(x_test[:num_examples])
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()
'''

Predicted val:  7.5485644
Actual val:  6.1398739967

Predicted val:  7.57099
Actual val:  5.6879263596

Predicted val:  8.126885
Actual val:  8.56275425608

Predicted val:  7.7047215
Actual val:  7.0988848364

Predicted val:  7.013243
Actual val:  6.66236955764

Predicted val:  7.4798093
Actual val:  7.054792384

Predicted val:  7.561883
Actual val:  8.68841774542

Predicted val:  6.583984
Actual val:  6.05609833714

Predicted val:  7.9347053
Actual val:  5.8642961692

Predicted val:  7.8661213
Actual val:  7.936641432

